In [ ]:
#
#Trabajo 2	– Preparación	de	texto,	representación	de	características	y	representación	de	documentos.
# 
# Erica Yubiana Herrera Sepulveda
# Juan Camilo Gómez Betancur
# Natalia Andrea Gaviria Angulo 
# 
# Universidad EAFIT 
# 2021-2
# 
# librerías en nltk, spacy, scikit-learn o gensim y 2) en pyspark, diferentes opciones de representación de características como one-bit, TF,TF-IDF, word2vec, doc2vec.

In [ ]:
#configuración en google colab
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# verificar que tengan instalado la librería 'pyspark'
!pip install pyspark

     |████████████████████████████████| 212.4 MB 56 kB/s 
     |████████████████████████████████| 198 kB 48.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=031ac84bf7b91af431c2c5f736291ad28edcdc162627dd77c42cec578361ea56
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
#Instalacion de nltk y de pyspark
!pip install nltk
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

#forma 1 de crear la sesión y el contexto Spark:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
#Se realiza la importacion de las dependencias a utilizar
#El	proyecto	lo	deberá	realizar	con: Python y	(nltk,	scikit	y/o	gensim) y PySpark,	con	Spark y	las	diferentes	librerías	de	nlp
import nltk
import gensim
import pandas as pd
import numpy as np
import re
import codecs
import matplotlib.pyplot as plt

In [ ]:
# directorios (path) de entrada y salida:
# Para saber donde estan mis datos de entrada, donde los voy a procesar y donde los voy a sacar, donde path_in es la ruta de donde se encuentran los datos de entrada
# path_out corresponde a la ruta de los datos de salida, filenametxt en este coloco el nombre del archivo el cual voy a procesar 
#Remoción	 de	 caracteres	 especiales	 y	 tokens	 que	 considere	 irrelevantes	 para	 una	consulta.
#
path_in="gdrive/MyDrive/st1800_20212/datasets/papers_sample_pdf/"

In [ ]:
archivo = !ls "gdrive/MyDrive/st1800_20212/datasets/papers_sample_pdf" | grep ".txt"

In [ ]:
archivo

['0704.3504.txt',
 '0706.1402.txt',
 '0710.0736.txt',
 '0803.2570.txt',
 '0808.0084.txt',
 '0811.1254.txt',
 '0811.2853.txt',
 '0812.2709.txt']

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(list(zip(archivo,archivos)), columns = ['archivo','contenido'])
print(df)

         archivo                                          contenido
0  0704.3504.txt  Smooth Rényi Entropy of Ergodic Quantum\nInfo...
1  0706.1402.txt  Analyzing Design Process and Experiments on th...
2  0710.0736.txt  1\n\nColour Image Segmentation by the\nVector-...
3  0803.2570.txt  1\n\nUnequal Error Protection:\nAn Information...
4  0808.0084.txt  On the hitting times of quantum versus random ...
5  0811.1254.txt  arXiv:0811.1254v1 [math.CO] 8 Nov 2008\n\nChap...
6  0811.2853.txt  Generating Random Networks Without Short Cycle...
7  0812.2709.txt  1\n\nVariations on a theme by Schalkwijk and K...


In [ ]:
#columns of dataframe
df

,archivo,contenido
0,0704.3504.txt,Smooth Rényi Entropy of Ergodic Quantum\nInfo...
1,0706.1402.txt,Analyzing Design Process and Experiments on th...
2,0710.0736.txt,1\n\nColour Image Segmentation by the\nVector-...
3,0803.2570.txt,1\n\nUnequal Error Protection:\nAn Information...
4,0808.0084.txt,On the hitting times of quantum versus random ...
5,0811.1254.txt,arXiv:0811.1254v1 [math.CO] 8 Nov 2008\n\nChap...
6,0811.2853.txt,Generating Random Networks Without Short Cycle...
7,0812.2709.txt,1\n\nVariations on a theme by Schalkwijk and K...


In [ ]:
#check number of columns
len(df.columns)

2

In [ ]:
#number of records in dataframe
df.columns

Index(['archivo', 'contenido'], dtype='object')

In [ ]:
# forma de conjunto de datos
print((df.count(),len(df.columns)))

(archivo      8
contenido    8
dtype: int64, 2)


In [ ]:
sparkDF=spark.createDataFrame(df) 

In [ ]:
from pyspark.ml.feature import Tokenizer
tokenization=Tokenizer(inputCol='contenido',outputCol='tokens')
tokenized_df = tokenization.transform(sparkDF)
tokenized_df.show(5)

+-------------+--------------------+--------------------+
|      archivo|           contenido|              tokens|
+-------------+--------------------+--------------------+
|0704.3504.txt|Smooth Rényi Ent...|[smooth, rényi, ...|
|0706.1402.txt|Analyzing Design ...|[analyzing, desig...|
|0710.0736.txt|1

Colour Image S...|[1, , colour, ima...|
|0803.2570.txt|1

Unequal Error ...|[1, , unequal, er...|
|0808.0084.txt|On the hitting ti...|[on, the, hitting...|
+-------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
from pyspark.ml.feature import StopWordsRemover
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')
refined_df=stopword_removal.transform(tokenized_df)
refined_df.select(['tokens','refined_tokens']).show(5)

+--------------------+--------------------+
|              tokens|      refined_tokens|
+--------------------+--------------------+
|[smooth, rényi, ...|[smooth, rényi, ...|
|[analyzing, desig...|[analyzing, desig...|
|[1, , colour, ima...|[1, , colour, ima...|
|[1, , unequal, er...|[1, , unequal, er...|
|[on, the, hitting...|[hitting, times, ...|
+--------------------+--------------------+
only showing top 5 rows



In [ ]:
refined_df.columns

['archivo', 'contenido', 'tokens', 'refined_tokens']

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [ ]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_count_df = refined_df.withColumn("token_count", len_udf(col('refined_tokens')))

In [ ]:
refined_count_df.orderBy(rand()).show(10)

+-------------+--------------------+--------------------+--------------------+-----------+
|      archivo|           contenido|              tokens|      refined_tokens|token_count|
+-------------+--------------------+--------------------+--------------------+-----------+
|0811.1254.txt|arXiv:0811.1254v1...|[arxiv:0811.1254v...|[arxiv:0811.1254v...|      10952|
|0710.0736.txt|1

Colour Image S...|[1, , colour, ima...|[1, , colour, ima...|       5158|
|0706.1402.txt|Analyzing Design ...|[analyzing, desig...|[analyzing, desig...|       2505|
|0808.0084.txt|On the hitting ti...|[on, the, hitting...|[hitting, times, ...|       7404|
|0704.3504.txt|Smooth Rényi Ent...|[smooth, rényi, ...|[smooth, rényi, ...|       3122|
|0812.2709.txt|1

Variations on ...|[1, , variations,...|[1, , variations,...|       8392|
|0803.2570.txt|1

Unequal Error ...|[1, , unequal, er...|[1, , unequal, er...|      20369|
|0811.2853.txt|Generating Random...|[generating, rand...|[generating, rand...|      13224|

In [ ]:
# Count Vectorizer
from pyspark.ml.feature import CountVectorizer
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')
cv_df=count_vec.fit(refined_df).transform(refined_df)
cv_df.select(['refined_tokens','features']).show(4,False)
bow = count_vec.fit(refined_df).vocabulary
print(bow)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# TF with HashingTF
from pyspark.ml.feature import HashingTF
# podria utilizar numFeatures como el tamaño del Bag of Words:
l = len(bow)
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features',numFeatures=l)
#hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features',numFeatures=11)
# compare la salida e interprete con y sin numFeatures:
#hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

hashing_df=hashing_vec.transform(refined_df)
hashing_df.show(4)

+-------------+--------------------+--------------------+--------------------+--------------------+
|      archivo|           contenido|              tokens|      refined_tokens|         tf_features|
+-------------+--------------------+--------------------+--------------------+--------------------+
|0704.3504.txt|Smooth Rényi Ent...|[smooth, rényi, ...|[smooth, rényi, ...|(11890,[27,33,55,...|
|0706.1402.txt|Analyzing Design ...|[analyzing, desig...|[analyzing, desig...|(11890,[19,23,26,...|
|0710.0736.txt|1

Colour Image S...|[1, , colour, ima...|[1, , colour, ima...|(11890,[13,19,21,...|
|0803.2570.txt|1

Unequal Error ...|[1, , unequal, er...|[1, , unequal, er...|(11890,[5,7,8,19,...|
+-------------+--------------------+--------------------+--------------------+--------------------+
only showing top 4 rows



In [ ]:
from pyspark.ml.feature import IDF
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)
tf_idf_df.show(4)

+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      archivo|           contenido|              tokens|      refined_tokens|         tf_features|     tf_idf_features|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|0704.3504.txt|Smooth Rényi Ent...|[smooth, rényi, ...|[smooth, rényi, ...|(11890,[27,33,55,...|(11890,[27,33,55,...|
|0706.1402.txt|Analyzing Design ...|[analyzing, desig...|[analyzing, desig...|(11890,[19,23,26,...|(11890,[19,23,26,...|
|0710.0736.txt|1

Colour Image S...|[1, , colour, ima...|[1, , colour, ima...|(11890,[13,19,21,...|(11890,[13,19,21,...|
|0803.2570.txt|1

Unequal Error ...|[1, , unequal, er...|[1, , unequal, er...|(11890,[5,7,8,19,...|(11890,[5,7,8,19,...|
+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 4 rows

